# Installation & Set-up

In [1]:
pip install requests folium overpy geopandas


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import overpy
from shapely.geometry import Point


In [3]:
import folium

m = folium.Map(location=[-33.9249, 18.4241], zoom_start=10)
m


In [4]:
#Capetown

city_location = [-33.9249, 18.4241]

indoor_activities = 3
outdoor_activities = 3
healthcare = 2
educational = 1

#Density representation
# 1 = zero or 1 amenities per km2
# 2 = 2-4 amenities per km2
# 3 = 5-6 amenities per km2
# 4 = 6-8 amenities per km2
# 5 = 8+ amenities per km2

public_transport = True
private_transport = False
active_transport = False


#Capetown

city_location = [-33.9249, 18.4241]

indoor_activities = 3
outdoor_activities = 3
healthcare = 2
educational = 1

#Density representation
# 1 = zero or 1 amenities per km2
# 2 = 2-4 amenities per km2
# 3 = 5-6 amenities per km2
# 4 = 6-8 amenities per km2
# 5 = 8+ amenities per km2

public_transport = True
private_transport = True
active_transport = True


density_mapping = {
    1: (0, 0),
    2: (1, 2),
    3: (3, 4),
    4: (5, 6),
    5: (7, float('inf'))
}


# Amenities categorization
amenities_mapping = {
    'indoor_activities': ["cafe", "restaurant", "mall", "theatre", "cinema"],
    'outdoor_activities': ["park", "playground", "picnic_site", "recreation_ground"],
    'healthcare': ["hospital", "clinic", "pharmacy"],
    'educational': ["school", "college", "university"],
    'public_transport': ["bus_station", "train_station"],  # Public transport remains unchanged
    'private_transport': ["motorway", "trunk"],  # Close to main roads, highways
    'active_transport': ["cycleway", "footway"]  # Bike paths, Walk paths
}

ct_bbox = [-34.373, 18.322, -33.471, 18.987]


# Amenities categorization
amenities_mapping = {
    'indoor_activities': ["cafe", "restaurant", "mall", "theatre", "cinema"],
    'outdoor_activities': ["park", "playground", "picnic_site", "recreation_ground"],
    'healthcare': ["hospital", "clinic", "pharmacy"],
    'educational': ["school", "college", "university"],
    'public_transport': ["bus_station", "train_station"],  # Public transport remains unchanged
    'private_transport': ["motorway", "trunk"],  # Close to main roads, highways
    'active_transport': ["cycleway", "footway"]  # Bike paths, Walk paths
}

ct_bbox = [-34.373, 18.322, -33.471, 18.987]



# Input Data (testing)

In [5]:
capetown_suburbs = gpd.read_file('Official_Planning_Suburbs.geojson')


In [6]:
import overpy

def get_amenities(bbox, amenity_type):
    api = overpy.Overpass()
    # Constructing the Overpass QL query
    query = f"""
    [out:json];
    (
      node["amenity"="{amenity_type}"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["amenity"="{amenity_type}"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["amenity"="{amenity_type}"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
    );
    out center;
    """
    
    response = api.query(query)
    
    # Extract the node or center of way/relation
    amenities = []
    for node in response.nodes:
        amenities.append((node.lat, node.lon))
    for way in response.ways:
        amenities.append((way.center_lat, way.center_lon))
    for relation in response.relations:
        amenities.append((relation.center_lat, relation.center_lon))
    
    return amenities


In [7]:
# Switch over the categories
def fetch_amenities_for_category(category, importance):
    amenity_list = amenities_mapping.get(category, [])
    fetched_amenities = []
    
    for amenity in amenity_list:
        fetched_amenities.extend(get_amenities(ct_bbox, amenity))
        
    # Here, you'd analyze the density and possibly filter out certain amenities 
    # if they exceed or don't meet the density criteria.
    return fetched_amenities

In [8]:
indoor_amenities = fetch_amenities_for_category('indoor_activities', indoor_activities)

In [9]:
import folium

def plot_amenities_on_map(amenities, base_location, zoom_start=12):
    # Initialize the map
    m = folium.Map(location=base_location, zoom_start=zoom_start)
    
    # Add amenities to the map
    for amenity in amenities:
        folium.Marker(amenity).add_to(m)
    
    # Return the map
    return m

# Call the function
plot_amenities_on_map(indoor_amenities, city_location)


In [10]:
# 2. Calculate density for each suburb
def calculate_density(suburb_geom, category, importance):
    bbox = suburb_geom.bounds
    amenities = fetch_amenities_for_category(category, importance)
    
    # Filter amenities that lie within the suburb
    in_suburb = [amenity for amenity in amenities if suburb_geom.contains(Point(amenity[1], amenity[0]))]
    
    # Calculate the density
    area_km2 = suburb_geom.area / 1e6  # Convert area to square kilometers
    density = len(in_suburb) / area_km2
    
    return density

In [11]:
def suburbs_meeting_density_requirement(category, importance):
    min_density, max_density = density_mapping[importance]
    qualifying_suburbs = []

    for _, suburb in capetown_suburbs.iterrows():
        if min_density <= calculate_density(suburb['geometry'], category, importance) <= max_density:
            qualifying_suburbs.append(suburb)

    return gpd.GeoDataFrame(qualifying_suburbs)

In [12]:
def visualize_suburbs(geo_df):
    m = folium.Map(location=city_location, zoom_start=11)

    folium.GeoJson(geo_df).add_to(m)
    return 

In [13]:
# Example usage:
indoor_activity_suburbs = suburbs_meeting_density_requirement('indoor_activities', 3)


In [14]:
visualize_suburbs(indoor_activity_suburbs)

ValueError: Cannot render objects with any missing geometries: Empty GeoDataFrame
Columns: []
Index: []